# Import Dependencies

In [1]:
# Import Main Dependencies
import os, re, ffmpeg, whisper
from pytubefix import YouTube, Stream
from pytubefix.cli import on_progress
from pytubefix.innertube import _default_clients

# Import Other Dependencies
import torch
from tqdm.auto import tqdm

# Define Utilities

In [2]:
def sanitize_filename(filename: str) -> str:
    # Escape Double Quotes
    filename = filename.replace('"', '\\"')

    # Replace Invalid Characters with "_"
    invalid_chars = re.compile(r'[<>:"/\\|?*]')
    sanitized_filename = invalid_chars.sub("_", filename)

    return sanitized_filename
    
def read_unique_items_from_file(file: str) -> list:
    with open(file, "r") as f:
        return list(set(url.strip() for url in f.readlines() if url.strip()))

# Set Configurations

In [3]:
# File Names
yt_video_links_filename = "YouTube Video Links.txt"
transcript_sentences_filename = "transcript_sentences.csv"

# Folder Names
video_output_path = "Video"
audio_output_path = "Audio"
transcription_output_path = "Transcription"

# Boolean Flags
remove_video = True
remove_audio = True

# Additional Dependency Configurations
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

# Collect Data (YouTube Videos)

In [4]:
def download_youtube_video(video_filename: str, stream: Stream) -> tuple[str, str]:
    # Create Video Directory
    os.makedirs(video_output_path, exist_ok=True)
    
    # Set Path for Video File
    video_file = os.path.join(video_output_path, video_filename)
    
    # Delete Old Existing Video File (note: to clean any corrupted file)
    if os.path.exists(video_file):
        os.remove(video_file)
        
    # Download Video File
    print("") # Just New Line for Better Output
    print(f'Downloading (Video): {video_filename}')
    print("") # Just New Line for Better Output
    stream.download(output_path=video_output_path, filename=video_filename)
    print("") # Just New Line for Better Output
    print("") # Just New Line for Better Output
    
    # Return Video File and Name
    return video_file, video_filename

# Audio Extraction (Video to Audio)

In [5]:
def extract_audio_from_video(video_file: str, video_filename: str) -> tuple[str, str]:
    # Create the Audio Directory
    os.makedirs(audio_output_path, exist_ok=True)

    # Set Audio File Name ("[YouTube Video ID] [title].mp3")
    audio_filename = f'{os.path.splitext(video_filename)[0]}.mp3'

    # Set Path for Audio File
    audio_file = os.path.join(audio_output_path, audio_filename)
    
    # Delete Old Existing Audio File (note: to clean any corrupted file)
    if os.path.exists(audio_file):
        os.remove(audio_file)
    
    # Extract Audio File
    print(f'Extracting (Audio): {audio_filename}')
    print("") # Just New Line for Better Output
    (
        ffmpeg
        .input(video_file)
        .output(audio_file, format="mp3", acodec="libmp3lame", loglevel="info")
        .run(overwrite_output=True)
    )
    
    # Return Audio File and Name
    return audio_file, audio_filename

# Transcription (Audio to Text)

In [6]:
def transcribe_audio_to_text(audio_file: str, audio_filename: str):
    # Create the Transcription Directory
    os.makedirs(transcription_output_path, exist_ok=True)
    
    # Set Transcription File Name ("[YouTube Video ID] [title].txt")
    transcription_filename = f'{os.path.splitext(audio_filename)[0]}.txt'
    
    # Set Path for Transcription File
    transcription_file = os.path.join(transcription_output_path, transcription_filename)
            
    # Get/Download OpenAI Whisper Model
    """ 
    Models: 
        tiny, base, small, medium, large, turbo
    English-Only:
        tiny.en, base.en, small.en, medium.en
    
    Required VRAM:              Speed:
        1) 1GB - tiny, base         1) 10x - tiny
        2) 2GB - small              2) 8x - turbo
        3) 5GB - medium             3) 7x - base
        4) 6GB - turbo              4) 4x - small
        5) 10GB - large             5) 2x - medium
                                    6) 1x - large
    
    Quote from OpenAI: 
        - The .en models for English-only applications tend to perform better, especially for the tiny.en and base.en models.
        We observed that the difference becomes less significant for the small.en and medium.en models.
    
    Note: 4GB lang VRAM ko kaya small.en ginamit
    """  
    print(f'Transcribing (Text): {transcription_filename}')
    print("") # Just New Line for Better Output
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    model = whisper.load_model("small.en", device=device)
    
    # Transcribe Audio File (Saves Whole Text in Memory Before Disk to Avoid Corruption)
    result = model.transcribe(audio_file, fp16=False, verbose=False)
    try:
        with open(transcription_file, "w") as f:
            f.write(result["text"])
    except:
        if os.path.exists(transcription_file):
            os.remove(transcription_file)

# Execute Data Gathering

In [ ]:
yt_urls = read_unique_items_from_file(yt_video_links_filename)

with tqdm(total=len(yt_urls), desc="Getting YouTube URLs") as pbar:
    for index, url in enumerate(yt_urls):        
        try:
            current = f'{index+1}/{len(yt_urls)}'
    
            # Get Video Information
            yt = YouTube(
                url,
                use_oauth=True,
                allow_oauth_cache=True,
                on_progress_callback=on_progress
            )
            stream = yt.streams.get_audio_only()
            video_id = yt.video_id
            
            # Sanitize Video File Name and Add YouTube Video ID
            video_filename = f'[{video_id}] {sanitize_filename(stream.default_filename)}'
            video_id_pattern_in_filename = re.compile(r'\[(.*?)\]')
            
            # Get File Name Without Extension (e.g., ".mp4")
            filename = os.path.splitext(video_filename)[0]
            
            # Skip If Transcription with YouTube ID Already Exists
            has_transcription_file = False
            pbar.set_description(f'Checking Existing Transcriptions [{current} File]')
            if video_id and os.path.exists(transcription_output_path):
                for existing_transcription_filename in os.listdir(transcription_output_path):
                    if existing_transcription_filename == ".ipynb_checkpoints": continue
                    if (
                        video_id_pattern_in_filename.search(existing_transcription_filename)
                        and video_id_pattern_in_filename.search(existing_transcription_filename).group(1) == video_id
                    ): 
                        has_transcription_file = True
                        break
            if has_transcription_file:
                # Delete/Keep Video File
                if remove_video:
                    for existing_video_filename in os.listdir(video_output_path):
                        if existing_video_filename == ".ipynb_checkpoints": continue
                        if (
                            video_id_pattern_in_filename.search(existing_video_filename)
                            and video_id_pattern_in_filename.search(existing_video_filename).group(1) == video_id
                        ): os.remove(existing_video_filename)
                            
                # Delete/Keep Audio File
                if remove_audio:
                    for existing_audio_filename in os.listdir(audio_output_path):
                        if existing_audio_filename == ".ipynb_checkpoints": continue
                        if (
                            video_id_pattern_in_filename.search(existing_audio_filename)
                            and video_id_pattern_in_filename.search(existing_audio_filename).group(1) == video_id
                        ): os.remove(existing_audio_filename)
                            
                pbar.update(1)
                continue
                
            # Log YouTube URL being Processed
            print("") # Just New Line for Better Output
            print(f'Found YouTube Video (URL): {url}')
            
            # Download YouTube Video
            pbar.set_description(f'Downloading [{current} Video] ')
            video_file, video_filename = download_youtube_video(video_filename, stream)
            
            # Extract Audio from Video -> Delete/Keep Video File
            pbar.set_description(f'Extracting [{current} Audio]')
            audio_file, audio_filename = extract_audio_from_video(video_file, video_filename)
            if remove_video: os.remove(video_file)
            
            # Transcribe Audio to Text -> Delete/Keep Audio File
            pbar.set_description(f'Transcribing [{current} Text]')
            transcribe_audio_to_text(audio_file, audio_filename)
            if remove_audio: os.remove(audio_file)
                
            pbar.update(1)
        except Exception as e: 
            print(f'{e}: {url}')
            
            pbar.update(1)
            
    pbar.set_description("Finished Data Gathering")

Getting YouTube URLs:   0%|          | 0/269 [00:00<?, ?it/s]


Found YouTube Video (URL): https://www.youtube.com/watch?v=YBMP19BF26w


 ↳ |█████████████████████████████████████████| 100.0%

Extracting (Audio): [YBMP19BF26w] Democrats DOMINATING Early Voting in Pennsylvania! Can Kamala Harris win the state.mp3

Transcribing (Text): [YBMP19BF26w] Democrats DOMINATING Early Voting in Pennsylvania! Can Kamala Harris win the state.txt



C:\Users\MSI Laptop\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(


Found YouTube Video (URL): https://www.youtube.com/watch?v=v-mDyQTwMe0


 ↳ |█████████████████████████████████████████| 100.0%

Extracting (Audio): [v-mDyQTwMe0] Trump Is GAINING In Pennsylvania!.mp3

Transcribing (Text): [v-mDyQTwMe0] Trump Is GAINING In Pennsylvania!.txt




100%|██████████████████████████▉| 85380/85674 [03:38<00:00, 390.38frames/s]


...


100%|███████████████████████████████████████████████████████████████████████| 92831/92831 [05:56<00:00, 260.66frames/s]



Found YouTube Video (URL): https://www.youtube.com/watch?v=UKoqg8OfDJU


 ↳ |██████████████████████████████████████████████████████████████████| 100.0%

Extracting (Audio): [UKoqg8OfDJU] 'Kamala Harris Sucks, Right_'_ Kristi Noem Blasts VP At Trump Town Hall In Oaks, Pennsylvania.mp3

Transcribing (Text): [UKoqg8OfDJU] 'Kamala Harris Sucks, Right_'_ Kristi Noem Blasts VP At Trump Town Hall In Oaks, Pennsylvania.txt




100%|███████████████████████████████████████████████████████████████████████| 29229/29229 [01:30<00:00, 322.96frames/s]



Found YouTube Video (URL): https://www.youtube.com/watch?v=jGQphgdPlNk


 ↳ |██████████████████████████████████████████████████████████████████| 100.0%

Extracting (Audio): [jGQphgdPlNk] Pennsylvania RALLY Reveals TRUMP'S 10 Shocking Promises.mp3

Transcribing (Text): [jGQphgdPlNk] Pennsylvania RALLY Reveals TRUMP'S 10 Shocking Promises.txt




 90%|████████████████████████████████████████████████████████████████▏      | 48136/53248 [02:27<00:19, 268.82frames/s]